<a href="https://colab.research.google.com/github/CostiD/CR39_Geant4/blob/main/CR39_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🔬 CR-39 Proton Irradiation — GEANT4 Simulation
**2.5 MeV protons | Detector at 3 cm | CR-39 Nuclear Track Detector**

This notebook:
1. Installs GEANT4 via Conda/Mamba
2. Clones and compiles the simulation from GitHub
3. Runs 50 000 proton events
4. Produces publication-quality figures

> **Runtime**: ~25 min total (mostly GEANT4 install). Use **Runtime → Change runtime type → T4 GPU** (or CPU) before starting.

In [ ]:
%%bash
wget -q https://github.com/conda-forge/miniforge/releases/latest/download/Miniforge3-Linux-x86_64.sh
bash Miniforge3-Linux-x86_64.sh -b -p /opt/mambaforge
/opt/mambaforge/bin/conda init bash
export PATH=/opt/mambaforge/bin:$PATH
mamba --version
echo "Mamba ready!"

## Step 1 — Install Mamba (fast conda) & GEANT4

In [ ]:
%%bash
export PATH=/opt/mambaforge/bin:$PATH
# Install GEANT4 (headless — no OpenGL needed for batch)
mamba install -y -c conda-forge geant4=11.1 cmake make gcc gxx 2>&1 | tail -20
echo 'GEANT4 installed!'

## Step 2 — Clone repository from GitHub

In [ ]:
# ⚠️  Change this to your actual GitHub repo URL after pushing!
REPO_URL = 'https://github.com/YOUR_USERNAME/CR39_Geant4.git'

import subprocess, os
result = subprocess.run(['git', 'clone', REPO_URL, '/content/CR39_Geant4'],
                        capture_output=True, text=True)
print(result.stdout or result.stderr)

## Step 3 — Compile the simulation

In [ ]:
%%bash
export PATH=/opt/mambaforge/bin:$PATH
# Source GEANT4 environment
source $(geant4-config --prefix)/bin/geant4.sh 2>/dev/null || \
  source /opt/mambaforge/share/Geant4/geant4make/geant4make.sh 2>/dev/null || true

mkdir -p /content/build && cd /content/build
cmake /content/CR39_Geant4 \
  -DCMAKE_PREFIX_PATH=/opt/mambaforge \
  -DCMAKE_BUILD_TYPE=Release \
  -DGeant4_DIR=$(geant4-config --prefix)/lib/Geant4-*/  2>&1 | tail -15
make -j$(nproc) 2>&1 | tail -20
ls -lh CR39_sim && echo 'Build SUCCESS!'

## Step 4 — Run the simulation (50 000 events)

In [ ]:
%%bash
export PATH=/opt/mambaforge/bin:$PATH
source $(geant4-config --prefix)/bin/geant4.sh 2>/dev/null || true
cd /content/build
cp /content/CR39_Geant4/run.mac .
time ./CR39_sim run.mac
echo "---"
head -5 cr39_output.csv
wc -l cr39_output.csv

## Step 5 — Publication-quality visualisation

In [ ]:
!pip install -q scipy matplotlib pandas
import shutil
shutil.copy('/content/build/cr39_output.csv', '/content/cr39_output.csv')

In [ ]:
import os
os.chdir('/content')
exec(open('/content/CR39_Geant4/analysis/visualize.py').read())

## Step 6 — Download results

In [ ]:
from google.colab import files
files.download('/content/cr39_output.csv')
files.download('/content/cr39_proton_analysis.pdf')
files.download('/content/cr39_proton_analysis.png')